# Forward dPL Multi-layer Terzaghi's 1D Consolidation Model

-- Land subsidence project: 15 March 2025 --

---


#### 1. Data Preperations

Essentially, the internals of what will eventually be the data loader for this model.

In [1]:
### Load in data
import sys
import os
import numpy as np
import torch
import pickle

sys.path.append('../../')
sys.path.append('../../dMG')  # Add the dMG root directory.
sys.path.append(os.path.abspath('..'))  # Add the parent directory of `scripts` to the path.

from scripts import load_config


#------------------------------------------#
# Define model settings here.
CONFIG_PATH = '/projects/mhpi/leoglonz/dPLT/src/dMG/conf/config_ls.yaml'
TEST_SPLIT = 0.2
#------------------------------------------#



config = load_config(CONFIG_PATH)


# Load data
with open(config['observations']['train_path'], 'rb') as f:
    data_dict = pickle.load(f)


# Normalize attributes for NN:
attrs = data_dict['attributes']
for i in range(attrs.shape[-1]):
    attrs[:, :, i] = (attrs[:, :, i] - attrs[:, :, i].mean()) \
        / attrs[:, :, i].std()
    
data_dict['xc_nn_norm'] = attrs


# Train-test split + convert to torch tensors
train_dataset = {}
test_dataset = {}
split = int(len(data_dict['forcing']) * (1 - TEST_SPLIT))

for key in data_dict.keys():
    train_dataset[key] = torch.tensor(
        data_dict[key][:split,],
        dtype=config['dtype'],
        device=config['device'],
    )
    test_dataset[key] = torch.tensor(
        data_dict[key][split:,],
        dtype=config['dtype'],
        device=config['device'],
    )

# Reshape to 3d
shape = train_dataset['xc_nn_norm'].shape
train_dataset['xc_nn_norm'] = train_dataset['xc_nn_norm'].reshape(
    shape[0], 
    shape[1],
    shape[2] * shape[3],
)

shape = test_dataset['xc_nn_norm'].shape
test_dataset['xc_nn_norm'] = test_dataset['xc_nn_norm'].reshape(
    shape[0], 
    shape[1],
    shape[2] * shape[3],
)   

HydroDL2 not found. Continuing without it.


#### 2. Load dPL Scheme 

Physical model (Terzaghi's equation) + LSTM


In [2]:
import importlib

from dMG import load_nn_model
from src.dMG.models.phy_models.terzaghi import TerzaghiMultiLayer as dPLT
importlib.reload(sys.modules['src.dMG.models.phy_models.terzaghi'])
importlib.reload(sys.modules['dMG'])


model = dPLT(config['dpl_model']['phy_model'], device=config['device'])
nn = load_nn_model(
    model,
    config['dpl_model'],
    device=config['device'],
)

#### 3. Forward Model


In [3]:
parameters = nn(train_dataset['xc_nn_norm'])

print(nn)
print(f"Input shape {train_dataset['xc_nn_norm'].shape}")
print(f"Output shape {parameters.shape}")

# predictions = model(
#     train_dataset,
#     parameters,
# )

LstmModel(
  (linear_in): Linear(in_features=81, out_features=256, bias=True)
  (lstm): Lstm(
    (lstm): LSTM(256, 256)
  )
  (linear_out): Linear(in_features=256, out_features=1, bias=True)
)
Input shape torch.Size([109, 16, 81])
Output shape torch.Size([109, 16, 1])


/projects/mhpi/leoglonz/dPLT/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(


In [21]:
parameters.shape

torch.Size([109, 16, 8])

In [4]:
train_dataset['attributes'].shape

torch.Size([109, 16, 9, 9])

suppose i have raw_parameters = x_dict['attributes'] with shape [time, sites, layers, parameters, 16] (repeated 16 times). I have another set of parameters which are learned from an lstm, parameters, but this only contains a certain set of parameters learned for a certain set of layers. For example, if i learned the first two parameters for layers 0 and 1, this would look like parameters ~ raw_parameters[time, sites, :2, :2, 16], where the 5th dimension is 16 because I learn the parameter 16 different times and take the average to reduce variance. This is represented by self.nmul in my code. Knowing this, can you modify the parameter handling functions in my model class to overwrite raw_parameters with parameters?

## 2. Terzaghi’s Consolidation Theory

### 2.1 Primary Consolidation Settlement

To quantify soil settlement due to groundwater drawdown, we apply Terzaghi’s one-dimensional consolidation theory. The analysis considers individual soil layers, each characterized by initial stress conditions, compressibility properties, and drainage constraints. The total primary consolidation settlement for a given layer depends on whether the soil is normally consolidated or overconsolidated.

For a normally consolidated soil layer where the final effective stress (\(\sigma'_f\)) does not exceed the preconsolidation pressure (\(\sigma'_p\)):

\[
S_i = \frac{H_i}{1 + e_{0i}} \times C_r \times \log_{10} \left(\frac{\sigma'_f}{\sigma'_0}\right)
\]

where \(H_i\) is the initial layer thickness, \(e_{0i}\) is the initial void ratio, \(C_r\) is the recompression index, and \(\sigma'_0\) is the initial effective stress.

For overconsolidated soils where \(\sigma'_f > \sigma'_p\), settlement is computed using:

\[
S_i = \frac{H_i}{1 + e_{0i}} \times \left[C_r \times \log_{10} \left(\frac{\sigma'_p}{\sigma'_0}\right) + C_c \times \log_{10} \left(\frac{\sigma'_f}{\sigma'_p}\right)\right]
\]

where \(C_c\) is the compression index and \(\sigma'_p = \text{OCR} \times \sigma'_0\) represents the preconsolidation pressure, defined by the overconsolidation ratio (OCR).

### 2.2 Time-Dependent Consolidation

Consolidation settlement occurs over time as excess pore water pressure dissipates. The rate of settlement is governed by the coefficient of consolidation (\(c_v\)):

\[
c_v = \frac{k}{\gamma_w \times m_v}
\]

where \(k\) is hydraulic conductivity, \(\gamma_w\) is the unit weight of water, and \(m_v\) is the coefficient of volume compressibility. The time-dependent settlement is described using the dimensionless time factor \(T_v\):

\[
T_v = \frac{c_v t}{H^2}
\]

where \(H\) is the drainage path length. The degree of consolidation \(U(t)\) is determined based on \(T_v\), with separate expressions for early- and late-stage consolidation:

- For \(T_v \leq 0.2\):
  \[
  U(t) = \frac{2}{\sqrt{\pi}} \times \sqrt{T_v}
  \]

- For \(T_v > 0.2\):
  \[
  U(t) = 1 - \frac{8}{\pi^2} \times \exp\left(-\frac{\pi^2 T_v}{4}\right)
  \]

Applying this time-dependent factor, the settlement at any time \(t\) is given by:

\[
S(x,y,t) = \sum_i S_i \times U(t)
\]

where \(S_i\) represents the primary consolidation settlement for layer \(i\). This framework allows for the integration of groundwater-induced stress changes with soil consolidation behavior, providing a time-dependent estimate of settlement at a given location.



Here's the updated methods section to better match your implementation:  

- I incorporated the details of `_init_layer_weights`, `_calculate_compression`, `PBM`, and the `forward` method, ensuring they align with how you're computing layer weights, compression, and subsidence.  
- I also included the historical groundwater effect adjustment in `PBM` and how layer weights factor into the subsidence calculation.  

Let me know if you want any refinements!  

---

### Methods

The TCM model simulates vertical subsidence due to changes in groundwater levels by integrating soil compressibility, effective stress changes, and historical groundwater variations. The model operates in a structured manner, processing input forcing data and learned parameters to estimate displacement.  

#### Layer Weight Initialization  

Each soil layer contributes differently to the overall subsidence, depending on its compressibility, fine fraction, and depth. The layer weights are computed as:  

\[
w_i = \text{compressibility}_i \times \text{fine fraction}_i \times e^{-\frac{i}{N}}
\]

where \( i \) is the layer index, \( N \) is the total number of layers, and the fine fraction is derived from the soil's coarse fraction. These weights are then normalized to ensure they sum to one across layers.  

#### Stress-Dependent Compression  

For each layer, compression is computed based on the change in effective stress, incorporating preconsolidation effects. The stress values are clamped to a minimum threshold to maintain numerical stability. The compression follows two regimes:  

- **Elastic compression** (when final stress remains below the preconsolidation pressure):  

\[
C = \frac{\text{parLT} \times \text{parRC}}{1 + \text{parVR}} \log \left(\frac{\sigma_f}{\sigma_0}\right)
\]

- **Plastic compression** (when stress exceeds preconsolidation pressure):  

\[
C = \frac{\text{thickness}}{1 + \text{parVR}} \left( \text{parRC} \log \left(\frac{\sigma_p}{\sigma_0}\right) + \text{parVCI} \log \left(\frac{\sigma_f}{\sigma_p}\right) \right)
\]

where \( \sigma_0 \), \( \sigma_f \), and \( \sigma_p \) are the initial, final, and preconsolidation pressures, respectively. Compression is further weighted by the soil's fine fraction.  

#### Historical Groundwater Effect  

The model incorporates a historical groundwater effect by computing a weighted sum of past groundwater levels, scaled by an exponential decay factor. This correction is added to the current groundwater level to capture lagged subsidence effects.  

#### Forward Pass  

For each site, the model:  
1. Unpacks forcing data and learned parameters.  
2. Computes layer weights using soil compressibility, texture, and depth factors.  
3. Iterates over time steps, calculating stress changes and compression for each layer.  
4. Aggregates the weighted layer compressions to compute total vertical displacement.  

The model outputs a time series of vertical displacement predictions across sites.  

---

Would you like me to add more equations or details?

## 2. Terzaghi’s Consolidation Theory

### 2.1 Primary Consolidation Settlement

To quantify soil settlement due to groundwater drawdown, we apply Terzaghi’s one-dimensional consolidation theory. The analysis considers individual soil layers, each characterized by initial stress conditions, compressibility properties, and drainage constraints. The total primary consolidation settlement for a given layer depends on whether the soil is normally consolidated or overconsolidated.

For a normally consolidated soil layer where the final effective stress (\(\sigma'_f\)) does not exceed the preconsolidation pressure (\(\sigma'_p\)):

\[
S_i = \frac{H_i}{1 + e_{0i}} \times C_r \times \log_{10} \left(\frac{\sigma'_f}{\sigma'_0}\right)
\]

where \(H_i\) is the initial layer thickness, \(e_{0i}\) is the initial void ratio, \(C_r\) is the recompression index, and \(\sigma'_0\) is the initial effective stress.

For overconsolidated soils where \(\sigma'_f > \sigma'_p\), settlement is computed using:

\[
S_i = \frac{H_i}{1 + e_{0i}} \times \left[C_r \times \log_{10} \left(\frac{\sigma'_p}{\sigma'_0}\right) + C_c \times \log_{10} \left(\frac{\sigma'_f}{\sigma'_p}\right)\right]
\]

where \(C_c\) is the compression index and \(\sigma'_p = \text{OCR} \times \sigma'_0\) represents the preconsolidation pressure, defined by the overconsolidation ratio (OCR).

The settlement contributions of individual layers are weighted based on their compressibility and fine fraction content, with an exponential depth scaling factor:

\[
w_i = \text{compressibility}_i \times \text{fine fraction}_i \times e^{-\frac{i}{N}}
\]

where \(i\) is the layer index and \(N\) is the total number of layers. These weights normalize the contributions of different layers, ensuring that subsidence is distributed according to soil properties.

### 2.2 Time-Dependent Consolidation

Consolidation settlement occurs over time as excess pore water pressure dissipates. The rate of settlement is governed by the coefficient of consolidation (\(c_v\)):

\[
c_v = \frac{k}{\gamma_w \times m_v}
\]

where \(k\) is hydraulic conductivity, \(\gamma_w\) is the unit weight of water, and \(m_v\) is the coefficient of volume compressibility. The time-dependent settlement is described using the dimensionless time factor \(T_v\):

\[
T_v = \frac{c_v t}{H^2}
\]

where \(H\) is the drainage path length. The degree of consolidation \(U(t)\) is determined based on \(T_v\), with separate expressions for early- and late-stage consolidation:

- For \(T_v \leq 0.2\):
  \[
  U(t) = \frac{2}{\sqrt{\pi}} \times \sqrt{T_v}
  \]

- For \(T_v > 0.2\):
  \[
  U(t) = 1 - \frac{8}{\pi^2} \times \exp\left(-\frac{\pi^2 T_v}{4}\right)
  \]

Applying this time-dependent factor, the settlement at any time \(t\) is given by:

\[
S(x,y,t) = \sum_i S_i \times U(t)
\]

where \(S_i\) represents the primary consolidation settlement for layer \(i\). This framework allows for the integration of groundwater-induced stress changes with soil consolidation behavior, providing a time-dependent estimate of settlement at a given location.

### 2.3 Stress-Dependent Compression and Historical Groundwater Effects

Stress-dependent compression accounts for both elastic and plastic regimes:

- **Elastic compression** (when \(\sigma'_f\) remains below \(\sigma'_p\)):
  
  \[
  C = \frac{\text{parLT} \times \text{parRC}}{1 + \text{parVR}} \log \left(\frac{\sigma'_f}{\sigma'_0}\right)
  \]

- **Plastic compression** (when \(\sigma'_f > \sigma'_p\)):
  
  \[
  C = \frac{\text{thickness}}{1 + \text{parVR}} \left( \text{parRC} \log \left(\frac{\sigma'_p}{\sigma'_0}\right) + \text{parVCI} \log \left(\frac{\sigma'_f}{\sigma'_p}\right) \right)
  \]

where \(\sigma'_0\), \(\sigma'_f\), and \(\sigma'_p\) are the initial, final, and preconsolidation pressures, respectively. Compression is further weighted by the soil's fine fraction.

To capture historical groundwater effects, past groundwater levels are integrated using an exponential decay factor, adjusting stress calculations to reflect lagged responses in soil behavior. This historical adjustment ensures that groundwater fluctuations over time are properly accounted for in subsidence predictions.

